<a href="https://colab.research.google.com/github/igvog/hadoop_learning/blob/main/halyk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import psycopg2 as pg
from sshtunnel import SSHTunnelForwarder
import pandas as pd
import mysql.connector
from mysql.connector import Error
from sqlalchemy import create_engine
import logging
import time
import warnings

In [19]:
pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [21]:
logging.basicConfig(filename=time.strftime("%Y%m%d%H%M%S")+'.log',
                    filemode='a',
                    format='%(asctime)s %(name)s %(levelname)s %(message)s',
                    datefmt='%H:%M:%S',
                    level=logging.DEBUG)

warnings.filterwarnings('ignore')

logging.info("Migration starting")

In [22]:
def my_sql_con():
    try:
        conn = mysql.connector.connect(host='halyk-sk.nidge.kz',
                                database='halyk',
                                user='export',
                                password='picass07')
        if conn.is_connected():
            logging.info("You're connected to MySQL")
            return conn
    except Error as e:
        logging.info(f"Print your error msg {e}")


def my_sql_con_ssh():
    try:
        logging.info('Connecting to the PostgreSQL Database...')
        ssh_tunnel = SSHTunnelForwarder(
                ('185.102.75.55'),
                ssh_username="localadmin",
                ssh_password= 'QAZqaz!@#123',
                remote_bind_address=('192.168.0.124', 3306)
            )
        ssh_tunnel.start()  
        engine = create_engine('mysql+pymysql://{user}:{password}@{host}:{port}/{db}'.format(
                 host='127.0.0.1',
                 port=ssh_tunnel.local_bind_port,
                 user='larauser',
                 password='AqrbdgCZq8e29mzS',
                 db='laradb'
                ))
        return engine
    except Error as e:
        logging.info(f"Connection Has Failed... {e}")


def pq_sql_con():
    try:
        logging.info('Connecting to the PostgreSQL Database...')
        ssh_tunnel = SSHTunnelForwarder(
                ('31.31.218.102'),
                ssh_username="devterminal",
                ssh_password= 'DevTerminal!@#$1234',
                remote_bind_address=('10.186.29.36', 5000)
            )
        ssh_tunnel.start()  
        #conn = pg.connect(
        #        host='127.0.0.1',
        #        port=ssh_tunnel.local_bind_port,
        #        user='devops',
        #        password= 'jgdnyie0J7ff',
        #        database='backend'
        #    )
        # DEV 
        # user='devops',
        # password='jgdnyie0J7ff',
        # remote_bind_address=('10.186.29.32', 5432)
        # PROD
        # backendprod
        # jgn0Kg*htyJ-0ghYV
        # remote_bind_address=('10.186.29.36', 5000)
        engine = create_engine('postgresql://{user}:{password}@{host}:{port}/{db}'.format(
                 host='127.0.0.1',
                 port=ssh_tunnel.local_bind_port,
                 user='backendprod',
                 password='jgn0Kg*htyJ-0ghYV',
                 db='backend'
                ))
        return engine
    except Error as e:
        logging.info(f"Connection Has Failed... {e}")


def check_pg_count(table_name: str) -> int:
    truncate_sql = f"select count(*) from {table_name}"
    result_dataFrame = pd.read_sql(truncate_sql, pq_con)
    return result_dataFrame.to_dict()['count'][0]


In [23]:
mysql_con = my_sql_con_ssh()
pq_con = pq_sql_con()

In [24]:
pk_tables_query = """SELECT "insurance_types" AS REFERENCED_TABLE_NAME
UNION
SELECT REFERENCED_TABLE_NAME
       FROM INFORMATION_SCHEMA.KEY_COLUMN_USAGE
       WHERE REFERENCED_TABLE_SCHEMA IS NOT NULL;"""
pk_tbl_dataframe = pd.read_sql(pk_tables_query, mysql_con)

all_tables_query = "SELECT TABLE_NAME FROM information_schema.tables WHERE TABLE_SCHEMA = 'laradb' order by TABLE_NAME;"
all_tbl_dataframe = pd.read_sql(all_tables_query, mysql_con)

skip_tables = ["policy_drafts", "personal_access_tokens"]
# policy_drafts uuid, ip_regions inet,
logging.info(f"Starting REFERENCED_TABLE_NAME")

In [25]:
all_tbl_dataframe

,TABLE_NAME
0,activations
1,admin_activations
2,admin_password_resets
3,admin_users
4,banners_regions
...,...
57,travel_programs
58,users
59,web_sliders
60,whatsapp_texts


In [51]:
df = pd.read_sql("SELECT * FROM users where id = '1435773'", mysql_con)
print("users")
df.to_sql(con=pq_con, name="users", schema="public", if_exists='append',index=False)
print(f"That table is uploaded users tables count={len(df.index)}")


users
That table is uploaded users tables count=1


In [ ]:
for item in pk_tbl_dataframe.to_dict()["REFERENCED_TABLE_NAME"].values():
    try:
        if not item.startswith("telescope") and check_pg_count(item) == 0 and item not in skip_tables:
            print(f"Starting That table is {item}")
            for chunk_dataframe in pd.read_sql(f"SELECT * FROM {item}", mysql_con, chunksize=10000):
              chunk_dataframe.to_sql(con=pq_con, name=item, schema="public", if_exists='append',index=False, method="multi")
            #result_dataframe = pd.read_sql(f"SELECT * FROM {item}", mysql_con)
            #result_dataframe.to_sql(con=pq_con, name=item, schema="public", if_exists='append',index=False, method="multi")
              print(f"That table is uploaded {item} tables count={len(chunk_dataframe.index)}")
        else:
            print(f"That table is not empty skipted {item}")
    except Error as e:
        print(f"Loading Has Failed on table{item}... {e}")


That table is not empty skipted insurance_types
That table is not empty skipted insurance_web_products
That table is not empty skipted telescope_entries
That table is not empty skipted regions
That table is not empty skipted users
That table is not empty skipted insurance_products
That table is not empty skipted bonuses
That table is not empty skipted travel_programs
That table is not empty skipted main_page_banners
That table is not empty skipted ip_regions
That table is not empty skipted permissions
That table is not empty skipted policy_drafts
That table is not empty skipted stories
That table is not empty skipted roles


In [52]:
print(f"Starting OTHER_TABLE_NAME")
for item in all_tbl_dataframe.to_dict()["TABLE_NAME"].values():
    try:
        if not item.startswith("telescope") and check_pg_count(item) == 0 and item not in skip_tables:
            print(f"That table started {item}")
            for chunk_dataframe in pd.read_sql(f"SELECT * FROM {item}", mysql_con, chunksize=15000):
                chunk_dataframe.to_sql(con=pq_con, name=item, schema="public", if_exists='append',index=False, method="multi")
                print(f"That table is uploaded {item} tables count={len(chunk_dataframe.index)}")
        else:
            print(f"That table is not empty skipted {item}")
    except Error as e:
        print(f"Loading Has Failed on table{item}... {e}")

print("Migration Done")

Starting OTHER_TABLE_NAME
That table started activations
That table is uploaded activations tables count=0
That table started admin_activations
That table is uploaded admin_activations tables count=0
That table started admin_password_resets
That table is uploaded admin_password_resets tables count=0
That table is not empty skipted admin_users
That table is not empty skipted banners_regions
That table is not empty skipted bonuses
That table is not empty skipted bonus_insurance_product
That table started bots
That table is uploaded bots tables count=0
That table is not empty skipted branches
That table is not empty skipted chatbot_flows
That table is not empty skipted db_notifications
That table is not empty skipted error_logs
That table started failed_jobs
That table is uploaded failed_jobs tables count=0
That table is not empty skipted faqs
That table is not empty skipted feedback
That table is not empty skipted insurance_accidents
That table is not empty skipted insurance_cases
That t

IntegrityError: ignored